- corrected_labels_created - 17/05/25

# Approach 2 - method 2
- Precisely crop the anomaly from the detections by syncing the subtrace before and after the anomaly w.r.t ref_samples
- to keep the lenght of feature vector same, we pad the features with trailing zeros to get length of 500 (max length of detection)
- The feature extraction is the dependent on the corresponding normal behaviour subtrace
- We tested this approach across all applications


In [ ]:
import json
import os
import sys
sys.path.append('../')  ### to detect libraries in the parent directory
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from collections import defaultdict
from libraries.utils import *
from libraries.exeint import exeInt
import plotly.express as px
from statistics import mode

# ############ configuration - trace ################
# ############################################


CODE = 'theft_protection'       ### application (code)       ###  'theft_protection', 'mamba2', 'lora_ducy'
BEHAVIOUR_FAULTY = 'faulty_data'            ### normal, faulty_data
BEHAVIOUR_NORMAL = 'normal'            ### normal, faulty_data
THREAD = 'single'           ### single, multi
VER = 4                     ### format of data collection
WINDOW = 500                 ### window size for subsequence
SUBSEQ =  'diag_subseq'        # 'diag_subseq'  , diag_subseq_multi      ### subsequence type, diag_subseq, subseq
OVERHEAD_LEN = 0          ### if 0 then ref_samples with complete train data considered

base_dir = '../../trace_data' ### can be replaced with 'csv', 'exe_plot', 'histogram'
normalbase_path = base_dir+f'/{CODE}/{THREAD}_thread/version_{VER}/{BEHAVIOUR_NORMAL}'
faultybase_path = base_dir+f'/{CODE}/{THREAD}_thread/version_{VER}/{BEHAVIOUR_FAULTY}'

print(normalbase_path)
print(faultybase_path)


################# configuration - diag ################
IS_VAR_WINDOW = False             ### True, False; wether to use variable window size or not

#####################################################

if OVERHEAD_LEN == 0:
    ref_samples_basepath = os.path.join(normalbase_path, f'diag_refsamples{WINDOW}')
else:
    ref_samples_basepath =  os.path.join(normalbase_path, f'diag_refsamples{WINDOW}_len-test_{OVERHEAD_LEN}')

ref_var_samples_basepath = os.path.join(normalbase_path, 'diag_var_refsamples')
diag_subseq_basepath = os.path.join(faultybase_path, f'{SUBSEQ}/subseq')
diag_el_basepath = os.path.join(faultybase_path, f'{SUBSEQ}/el')
subseq_label_basepath = os.path.join(diag_subseq_basepath, 'subseq_labels')
test_labels_basepath = os.path.join(faultybase_path, 'labels')


print('ref_samples_path:\n', ref_samples_basepath)
print('ref_var_samples_path:\n', ref_var_samples_basepath)
print('diag_subseq_path:\n', diag_subseq_basepath)

######### get paths #######################
ref_samples_path = [os.path.join(ref_samples_basepath, x) for x in os.listdir(ref_samples_basepath)]
# ref_var_samples_path = [os.path.join(ref_var_samples_basepath, x) for x in os.listdir(ref_var_samples_basepath)]   

train_varlist_path = os.listdir(normalbase_path)
train_varlist_path = [os.path.join(normalbase_path, x) for x in train_varlist_path if 'varlist' in x]

######### get paths #######################
paths_log, paths_traces, varlist_path, paths_label = get_paths(faultybase_path)

test_subseq_path = [os.path.join(diag_subseq_basepath, x) for x in os.listdir(diag_subseq_basepath)]
test_feature_path = [os.path.join(diag_el_basepath, x) for x in os.listdir(diag_el_basepath)]
test_labels_path = [os.path.join(subseq_label_basepath, x) for x in os.listdir(subseq_label_basepath)]
eval_labels_path = [os.path.join(test_labels_basepath, x) for x in os.listdir(test_labels_basepath)]


# ### remove.Ds_store from all lists
train_varlist_path = [x for x in train_varlist_path if '.DS_Store' not in x]
varlist_path = [x for x in varlist_path if '.DS_Store' not in x]
paths_label = [x for x in paths_label if '.DS_Store' not in x]
ref_samples_path = [x for x in ref_samples_path if '.DS_Store' not in x]
# ref_var_samples_path = [x for x in ref_var_samples_path if '.DS_Store' not in x]
test_subseq_path = [x for x in test_subseq_path if '.DS_Store' not in x if '.json' in x]
test_feature_path = [x for x in test_feature_path if '.DS_Store' not in x]
test_labels_path = [x for x in test_labels_path if '.DS_Store' not in x]
eval_labels_path = [x for x in eval_labels_path if '.DS_Store' not in x]

varlist_path.sort()

# print(paths_log)
# print(paths_traces)
# print(varlist_path)
# print(paths_label)

if IS_VAR_WINDOW:
    # train_data_path = ref_var_samples_path
    raise ValueError('Ref samples for variable window missing')
else:
    train_data_path = ref_samples_path

test_data_path = test_subseq_path

# print('train_data:', train_data_path)
print('No. of ref samples', len(train_data_path))
# print('test_data:\n', test_data_path)
print(len(test_data_path))
print('test_labels:\n', test_labels_path)
print('eval_labels:\n', eval_labels_path)



In [ ]:
#### End to end overhead ####

import time
import psutil

# process = psutil.Process(os.getpid())
# start_mem = process.memory_info().rss / (1024 * 1024)  # in MB
# start_time = time.perf_counter() 

## Detect and Seperate if multiple instances

In [ ]:
def strip_correct_part(ref_samples, test_events, test_intervals):
    '''
    check if any matching event trace in present based on first 2 points
    if yes, then check the number of matching events and intervals, remove the matching part in the event trace and return the remaining part
    if no, then return the same event trace
    '''

    test_data_len = len(test_events)
    # print('test_events:', test_events)
    # print('test_data_len:', test_data_len)
    # print('ref_samples', ref_samples)
    ### shortlist the reference samples which has first 5 elements same as the test_trace
    shortlisted_ref_events = []
    shortlisted_ref_intervals = []
    zero_count = []
    for ref_sample in ref_samples:
        # print('ref_sample:', ref_sample[0][:5])
        # event_diff = np.array(ref_sample[0][0:len(test_events)]) - np.array(test_events)
        # print('event_diff:', event_diff)
        # print('event_diff:', len(event_diff))
        # print('zeros:', np.where(event_diff==0)[0].shape)
        # if len(test_events) == 276:
        #     print('ref_sample:', ref_sample[0][:5])
        #     print('test_events:', test_events[:5])
        if ref_sample[0][:2] == test_events[:2]:
            ref_events = ref_sample[0][:test_data_len]
            ref_intervals = ref_sample[1][:test_data_len]
            # print('ref_events', ref_events)
            if len(test_events) > 500:
                diff_events = np.array(ref_events) - np.array(test_events[:500])
                diff_intervals = np.abs(np.array(ref_intervals) - np.array(test_intervals[:500]))
            else:
                diff_events = np.array(ref_events) - np.array(test_events)
                diff_intervals = np.abs(np.array(ref_intervals) - np.array(test_intervals))
    
            # if len(test_events) == 276:
            #     print('diff_events:', diff_events)
            #     print('diff_intervals:', diff_intervals)
            # print('diff events', diff_events)
            count = 0
            # print(sf[0], sf[1])
            for esf, esi in zip(diff_events, diff_intervals):
                ### check if events and intervals are same
                # if esf == 0 and esi < 5:
                if esf == 0:
                    count += 1
                else:
                    break   ### part of the logic, do not remove

            # print('zero_count:', count)
            ### depulicate the ref samples
            if ref_events not in shortlisted_ref_events:
                zero_count.append(count)
                shortlisted_ref_events.append(ref_events)
                shortlisted_ref_intervals.append(ref_intervals)
            # print('count:', count)  

        # break

    # print('zero_count:', zero_count)
    # print('shortlisted_ref_samples:', len(shortlisted_ref_events))

    ### select the ref_sample_events with maximum leading zeros
    if len(zero_count) != 0:
        max_zero_count = max(zero_count)
        zero_count = np.array(zero_count)
        max_zero_count_ind = np.where(zero_count==max_zero_count)[0][0]
        # print('max_zero_count_ind:', max_zero_count_ind)
        selected_ref_events = shortlisted_ref_events[max_zero_count_ind]
        selected_ref_intervals = shortlisted_ref_intervals[max_zero_count_ind]
        # print('selected_ref_events:', selected_ref_events[:max_zero_count+1])
    else:
        max_zero_count = 0
        selected_ref_events = None
        selected_ref_intervals = None

    if max_zero_count == 0:
        print('func: No match found')
        return None, selected_ref_events, selected_ref_intervals
    else:
        ### select the point where the last match happened
        # last_matched_point = max_zero_count-1
        # print('last_matched_point:', last_matched_point)
        # print('test_events:', test_events[:last_matched_point])
        # striped_test_events = test_events[last_matched_point:]
        # striped_test_intervals = test_intervals[last_matched_point:]
        # striped_test_timestamps = test_timestamps[last_matched_point:]
        # print('striped_test_events:', striped_test_events)
        # print('max count:', max_zero_count, len(test_events), len(striped_test_events))

        
        if max_zero_count == len(test_events):
            print('func: All events are same')
            return max_zero_count, selected_ref_events, selected_ref_intervals
        else:
            return max_zero_count, selected_ref_events, selected_ref_intervals



In [ ]:
# !pip install memory_profiler

In [ ]:
%%time
%reload_ext memory_profiler

######## computation overhead ######
start_time = time.perf_counter() 
####################################

##########################################################
##########################################################
    
### load all the reference samples (fixed window size)
ref_samples = []
for ref_sample in train_data_path:
    ref_samples.append(read_traces(ref_sample))


#########################################################
#########################################################

### load the test samples and compare with the reference samples
anomaly_instances = []
anomaly_timestamps = []
anomaly_features = []
test_files = []
for test_data in test_data_path[0:]:
    print('test_data:', test_data)
    ### read the subseq
    test_trace = read_traces(test_data)
    # print('test_trace:', test_trace)
    test_data_len = len(test_trace)
    # print('test_data_len:', test_data_len)

    # if test_data_len > 500:
    #     # print('test data length is more than 500, skipping...')
    #     # missing_features.append((test_data, 'test data length is more than 500'))
    #     # continue

    #     print('test data length is more than 500, truncating...')
    #     test_trace = test_trace[:500]
    #     test_data_len = 500
    
    ### transform the test trace from [(var,ts1), (var,ts2), (var, ts3)] to [[var1, var2, var3], [ts1, ts2, ts3]]
    ### old implementation with 0 at start of intervals
    # test_events = []
    # test_intervals = []
    # test_timestamps = []
    # prev_time = test_trace[0][1]
    # time_diff = 0
    # for x in test_trace:
    #     time_diff = x[1] - prev_time
    #     test_intervals.append(time_diff)
    #     prev_time = x[1]
    #     test_events.append(x[0])
    #     test_timestamps.append(x[1])

    ### new implementation without 0 at start of intervals
    test_events = []
    test_intervals = []
    test_timestamps = []
    for x,y in zip(test_trace[:-1], test_trace[1:]):
        test_events.append(x[0])   ### first event
        test_intervals.append(y[1] - x[1])   ### difference between the timestamps of second and first event
        test_timestamps.append(x[1])   ### first timestamp
    print('test_events:', len(test_events))
    print('test_data_len', test_data_len)
    # print('test_intervals:', test_intervals)

    assert len(test_events)+1 == len(test_intervals)+1 == test_data_len

    # ### get detection class

    # test_class_labels = read_json(test_labels_path[0])

    ### store the first to consecutive points as the first anomaly instance
    an_instance = []
    an_timestamps = []
    all_instances = []
    all_timestamps = []
    all_ref_events = []
    all_ref_intervals = []
    all_striped_test_events = []
    all_striped_test_intervals = []
    all_striped_timestamps = []
    an_affected_events = []    ### store the additional/missing events compared to ref_samples
    all_affected_events = []
    striped_ref_events = []

    striped_test_events = test_events
    striped_test_intervals = test_intervals
    striped_timestamps = test_timestamps
    # print('striped_test_events:', len(striped_test_events))
    # print('striped_test_intervals:', striped_test_intervals, len(striped_test_intervals))
    has_checked = False
    max_zero_old_count = 0
    i = 0
    while len(striped_test_events) > 0:
    # while i < 6:
        ### remove the initial correct part of the trace
        # print('ref_samples:', ref_samples, striped_test_events, striped_test_intervals)
        print('give input:', len(striped_test_events),)
        max_zero_count, selected_ref_events, selected_ref_intervals = strip_correct_part(ref_samples, striped_test_events, striped_test_intervals)
        # print('max_zero_count:', max_zero_count, 'len:', len(striped_test_events))
        
        if selected_ref_events != None:
            if len(selected_ref_events) == len(striped_test_events):
                ### first collect the trace that is given as input
                all_striped_test_events.append(striped_test_events)
                all_striped_test_intervals.append(striped_test_intervals)
                all_striped_timestamps.append(striped_timestamps)
                ### get the ref trace that matched with the test trace
                all_ref_events.append(selected_ref_events)
                all_ref_intervals.append(selected_ref_intervals)
            else: 
                ### if the len ar not same, then test_events are longer than 500 (max length of ref sample)
                print('1', len(striped_test_events), len(selected_ref_events), len(striped_ref_events))
                all_striped_test_events.append(striped_test_events[:500])
                all_striped_test_intervals.append(striped_test_intervals[:500])
                all_striped_timestamps.append(striped_timestamps[:500])
                all_ref_events.append(selected_ref_events)
                all_ref_intervals.append(selected_ref_intervals)
                print('1.1', len(striped_test_events[:500]), len(selected_ref_events[:500]), len(striped_ref_events[:500]))
        else:
            print('No match found')
            print('2', len(striped_test_events), (len(selected_ref_events) if selected_ref_events != None else None), len(striped_ref_events))
            if len(striped_ref_events) == len(striped_test_events):
                all_striped_test_events.append(striped_test_events)
                all_striped_test_intervals.append(striped_test_intervals)
                all_striped_timestamps.append(striped_timestamps)
                all_ref_events.append(striped_ref_events)
            else:
                if len(striped_test_events) > 500:
                    all_striped_test_events.append(striped_test_events[:len(striped_ref_events)])
                    all_striped_test_intervals.append(striped_test_intervals[:len(striped_ref_events)])
                    all_striped_timestamps.append(striped_timestamps[:len(striped_ref_events)])
                    all_ref_events.append(striped_ref_events)
                    print('2.1', len(striped_test_events[:len(striped_ref_events)]), (len(selected_ref_events) if selected_ref_events != None else None), len(striped_ref_events))
                # else:
                    # raise ValueError('Mismatch in lengths of selected_ref_events and striped_test_events')

        # print('selected_ref_events:', selected_ref_events)

        if max_zero_count == len(striped_test_events):
            print('All events are same')
            striped_test_events = []
            striped_test_intervals = []
            striped_timestamps = []
        # elif max_zero_count != None:
        #     striped_test_events = striped_test_events[max_zero_count-1:]
        #     striped_test_intervals = striped_test_intervals[max_zero_count-1:]
        #     striped_timestamps = striped_timestamps[max_zero_count-1:]
        # print('3', len(striped_test_events), (len(selected_ref_events) if selected_ref_events != None else None), len(striped_ref_events))

        # break
        ### store the first anomaly instance (first two consecutive points)
        if max_zero_count != None:
            # print('debug', max_zero_count, len(striped_test_events))
            if len(striped_test_events) != 0:
                if has_checked:
                    if len(an_instance) != 0:
                        ### new anomaly instance detected
                        all_instances.append(an_instance)
                        all_timestamps.append(an_timestamps)
                        all_affected_events.append(an_affected_events)
                        an_instance = []
                        an_timestamps = []
                        an_affected_events = []

                    ### start, where first anomaly instance is detected
                    print('Checked, new instance created', 'old, new', max_zero_old_count, max_zero_count)
                    seq_start = np.clip(max_zero_count-2, 0, None)
                    seq_end = np.clip(seq_start+5, 0, len(striped_test_events))
                    # print('seq_start:', seq_start, 'seq_end:', seq_end)
                    an_instance.extend(striped_test_events[seq_start:seq_end])
                    an_timestamps.extend(striped_timestamps[seq_start:seq_end])
                    # print('selected_ref_events:', selected_ref_events)
                    an_affected_events.extend(selected_ref_events[seq_start:seq_end])
                    striped_test_events = striped_test_events[seq_end:]
                    striped_test_intervals = striped_test_intervals[seq_end:]
                    striped_timestamps = striped_timestamps[seq_end:]
                    striped_ref_events = selected_ref_events[seq_end:]
                    # print('an_instance:', an_instance)
                    print('striped_test_events 3.1:', len(striped_test_events), (len(selected_ref_events) if selected_ref_events != None else None), len(striped_ref_events))
                    has_checked = False
                else:
                    if len(an_instance) != 0:
                        ### new anomaly instance detected
                        all_instances.append(an_instance)
                        all_timestamps.append(an_timestamps)
                        all_affected_events.append(an_affected_events)
                        an_instance = []
                        an_timestamps = []
                        an_affected_events = []

                    ### recheck from last 5 matching points to see in really an anomaly
                    print('Detected discripancy, rechecking...')
                    max_zero_old_count = max_zero_count
                    step_back = np.clip(max_zero_count-30, 0, None)
                    print('steping back', step_back)
                    striped_test_events = striped_test_events[step_back:]
                    striped_test_intervals = striped_test_intervals[step_back:]
                    striped_timestamps = striped_timestamps[step_back:]
                    striped_ref_events = selected_ref_events[step_back:]
                    print('striped_test_events 2:', len(striped_test_events), (len(selected_ref_events) if selected_ref_events != None else None), len(striped_ref_events))
                    has_checked = True
                    continue ### continue to the next iteration

            else:
                if len(an_instance) != 0:
                    all_instances.append(an_instance)
                    all_timestamps.append(an_timestamps)
                    all_affected_events.append(an_affected_events)
                    an_instance = []
                    an_timestamps = []
                    an_affected_events = []
                anomaly_instances.append(all_instances)
                anomaly_timestamps.append(all_timestamps)
                anomaly_features.append(all_affected_events)
                test_files.append(test_data)
                print('DONE 1')
                break ### part of the logic, do not remove
        else:
            if len(striped_test_events) > 1:
                ### normal functionality loop
                print('extending previous instance')
                an_instance.extend(striped_test_events[:1])
                an_timestamps.extend(striped_timestamps[:1])
                an_affected_events.extend(striped_ref_events[:1])
                striped_test_events = striped_test_events[1:]
                striped_test_intervals = striped_test_intervals[1:]
                striped_timestamps = striped_timestamps[1:]
                striped_ref_events = striped_ref_events[1:]
                print('striped_test_events 3.2:', len(striped_test_events), (len(selected_ref_events) if selected_ref_events != None else None), len(striped_ref_events))
            else:
                ### for last iteration, when len(striped_test_events) == 1   
                if len(an_instance) != 0:
                    all_instances.append(an_instance)
                    all_timestamps.append(an_timestamps)
                    all_affected_events.append(an_affected_events)
                    an_instance = []
                    an_timestamps = []
                    an_affected_events = []
                anomaly_instances.append(all_instances)
                anomaly_timestamps.append(all_timestamps)
                anomaly_features.append(all_affected_events)
                test_files.append(test_data)
                print('DONE 2')
                break  #### part of the logic, do not remove
        print('an_instance:', an_instance)
        print('all_instances:', all_instances)
        print('an_affected_events:', an_affected_events)
        print('all_affected_events:', all_affected_events)
        if len(striped_test_events) == 0:
            all_instances.append(an_instance)
            all_timestamps.append(an_timestamps)
            all_affected_events.append(an_affected_events)
            an_instance = []
            an_timestamps = []
            an_affected_events = []
            anomaly_instances.append(all_instances)
            anomaly_timestamps.append(all_timestamps)
            anomaly_features.append(all_affected_events)
            test_files.append(test_data)
            print('DONE 3')
        print('')    
        i += 1
        # if i == 2:
        #     print('break')
        #     break
    
    print('all_instances:', all_instances)
    print('all_affected_events:', all_affected_events)
    print('-------\n')
    
    # break
# print('anomaly_instances:', anomaly_instances) 
# print('anomaly_timestamps:', anomaly_timestamps)
# print('anomaly_features:', anomaly_features)


######## computation overhead ######
# print(f"Memory used (Training):", memory_used)
# print(f"Memory used (Training):", round(np.mean(np.array(memory_used)), 2) )
print('overhead_len:', OVERHEAD_LEN)
end_time = time.perf_counter()
elapsed_ms = (end_time - start_time) * 1000
print(f"\nTime taken (Training): {elapsed_ms:.2f} ms")

%memit
####################################

### Plot Trace

In [ ]:
# import plotly.graph_objects as go
# FONTSIZE = 20
# PLOTWIDTH = 1500

# # # x_axis = np.arange(0, len(test_trace), 1)



# # ### prepare test_trace for plotting
# # plot_data = dict()
# # plot_data['subseq'] = test_events   ### y_data (traces)

# # # for i, fv in enumerate(shortlisted_ref_samples):
# # #     plot_data[f'feat1_{i}'] = fv[0]
# # plot_data['ref_samples'] = selected_ref_events
    
# # df_feat1 = pd.DataFrame(plot_data)

# # plot_data = dict()
# # plot_data['intervals'] = test_intervals   ### y_data (traces)

# # # for i, fv in enumerate(feature_vectors):
# # #     plot_data[f'feat2_{i}'] = fv[1]
# # plot_data['ref_intervals'] = selected_ref_intervals

# # df_feat2 = pd.DataFrame(plot_data)

# # fig = px.line(df_feat1, title='features')
# # fig.show()

# # fig = px.line(df_feat2, title='features')
# # fig.show()

# for i in range(len(all_striped_test_events)):
#     plot_data = dict()
#     plot_data['subseq'] = all_striped_test_events[i]   ### y_data (traces)
#     print('ref:', all_ref_events[i])
#     print('det:', all_striped_test_events[i])
#     print('len ref', len(all_ref_events[i]) if all_ref_events[i] != None else None)
#     print('len det', len(all_striped_test_events[i]) if all_striped_test_events[i] != None else None)
#     if all_ref_events[i] != None:
#         plot_data['ref_samples'] = all_ref_events[i]
            
#         df_feat1 = pd.DataFrame(plot_data)
#         # print('df_feat1:', df_feat1)
        
#         fig = go.Figure()
#         for col in df_feat1.columns:
#             fig.add_trace(go.Scatter(y=df_feat1[col], mode='lines+markers', name=col))

#         # fig = px.line(df_feat1, title='features')


#         #### formatting to match Dash board plots
#         fig.update_layout(
#             # title_text="Event Trace without Time",
#             xaxis=dict(
#                 tickfont = dict(size = FONTSIZE),
#                 # titlefont = dict(size = FONTSIZE),
#                 color='black',
#             ),
#             yaxis=dict(
#                 title="Events",
#                 tickfont = dict(size = FONTSIZE),
#                 # titlefont = dict(size = FONTSIZE),
#                 color='black',
#             ),
#             autosize=True,
#             width=PLOTWIDTH,
#             # height=PLOTHEIGHT,
#             font_size=FONTSIZE,
#             plot_bgcolor='rgba(0,0,0,0)',
            
#         )

#         fig.update_xaxes(
#             mirror=True,
#             ticks='outside',
#             showline=True,
#             linecolor='black',
#             gridcolor='lightgrey'
#         )
#         fig.update_yaxes(
#             mirror=True,
#             ticks='outside',
#             showline=True,
#             linecolor='black',
#             gridcolor='lightgrey',
#             autorange=True,
#         )

#         # style all the traces
#         fig.update_traces(
#             #hoverinfo="name+x+text",
#             line={"width": 0.5},
#             marker={"size": 8},
#             mode="lines+markers",
#             showlegend=True,
            
#         )
#         #############
        
#         fig.show()

#         # plot_data = dict()
#         # print('check:', len(all_striped_test_intervals[i]), len(all_ref_intervals[i]))
#         # plot_data['intervals'] = all_striped_test_intervals[i]   ### y_data (traces)
#         # plot_data['ref_intervals'] = all_ref_intervals[i]

#         # df_feat2 = pd.DataFrame(plot_data)

#         # fig = px.line(df_feat2, title='features')
#         # fig.show()

#     # break


In [ ]:
# for an in anomaly_instances:
#     print('an:', an)
#     print('len:', len(an))
#     print('')

In [ ]:
# test_class_labels = read_json(test_labels_path[0])
# # print(test_class_labels)

# for l, i, k, j in zip(test_files, anomaly_instances, anomaly_timestamps, anomaly_features):
#     # print(l)
#     file_name = l.split('/')[-1].split('.')[0]
#     print(file_name)

#     class_labels = test_class_labels[file_name]

#     print('length of labels and predictions:', len(class_labels), len(i))
#     print('class:', class_labels)
#     print('detection', i)
#     print('missing/additional events:', j)
#     print('timestamp', k)

#     print('')
#     # break

### Evaluation

In [ ]:
DIFF_VAL = 5

ei = exeInt()

#### check how many instance are identified correctly

### load labels for each trial file
all_test_labels = dict()
for label_path in eval_labels_path:
    # print(label_path)
    file_name = label_path.split('/')[-1].split('.')[0][:-7]
    print('file_name 1:', file_name)
    eval_labels = read_json(label_path)
    key = list(eval_labels['labels'].keys())[0]
    labels = eval_labels['labels'][key]
    all_test_labels[file_name] = labels
    # print('labels:', labels)
    # print(eval_labels)
    # print('')

inst_len_all = []
gt_in_inst_all = []
inst_in_gt_all = defaultdict(list)

all_tp = []
all_fp = []
all_fn = []
all_gt = []
all_detections = [] ### format [file1_detection, file2_detection] -> file1_detection: [(state1, 0), (ts1, ts2), filename]  
all_features = []  ### collection of features (corresponding events for anomaly from reference samples)
y_pred_all = []
y_true_all = []
correct = 0
incorrect = 0
for file, instances, inst_time, an_feature in zip(test_files, anomaly_instances, anomaly_timestamps, anomaly_features):
    print(file)
    file_name = file.split('/')[-1].split('.')[0].split('_')
    file_name = '_'.join(file_name[:-1])
    print('file_name 2:', file_name)
    det_file_name = file.split('/')[-1].split('.')[0]
    print('det_file_name:', det_file_name)
    subseq_ind = file.split('/')[-1].split('.')[0].split('_')[-1]
    subseq_start = int(subseq_ind.split('-')[0])
    subseq_end = int(subseq_ind.split('-')[1])
    label_file = file.split('/')[:-3]
    label_file = '/'.join(label_file) + '/labels/' + file_name + '_labels.json'
    test_file = file.split('/')[:-3]
    test_file = '/'.join(test_file) + '/' + file_name
    print('test_file:', test_file)
    print('label_file:', label_file)
    # print('subseq_ind:', subseq_ind)
    print('instances:', instances)
    print('inst_time:', inst_time)
    print('feature:', an_feature)
    
    ### check against labels if the instances are correct
    labels = all_test_labels[file_name]
    # print('instances:', instances)
    # print('inst_time:', inst_time)
    # print('labels:', labels)

    detection = []
    feature = []
    for i, t, k in zip(instances, inst_time, an_feature):
        print(i, t, k)
        start_anomaly = t[0]
        end_anomaly = t[-1]
        start_event = i[0]
        end_event = i[-1]

        anomaly = [(start_event, end_event), (start_anomaly, end_anomaly), det_file_name]
        detection.append(anomaly)
        feature.append(k)

        # print('anomaly and time:', i, t)

        ######### calculate avg length of instances ##########
        inst_len = len(i)
        # print('inst_len:', inst_len)
        inst_len_all.append(inst_len)

        ######## calculate number of anomalies (GT) present in each detection (metric to evaluate how well is the seperation done, ideally it should be 1) #######
        gt_in_inst = []
        for l in labels:
            gt_ind1, gt_ind2 = l[2], l[3]
            # print('gt:', gt_ind1, gt_ind2)
            # print('start_anomaly:', start_anomaly, 'end_anomaly:', end_anomaly)
            if (gt_ind1 >= start_anomaly and gt_ind1 < end_anomaly) or (gt_ind2 > start_anomaly and gt_ind2 <= end_anomaly) or (gt_ind1 < start_anomaly < gt_ind2) or (gt_ind1 < end_anomaly < gt_ind2):
                gt_in_inst += [l]
        # print('gt_in_inst:', gt_in_inst)
        num_gt_in_inst = len(gt_in_inst)
        if num_gt_in_inst > 0:
            gt_in_inst_all.append(num_gt_in_inst)
            for gtii in gt_in_inst:
                inst_key = str(gtii)
                inst_in_gt_all[inst_key].append(anomaly)


    gts = []
    for l in labels:
        # print('l:', l)
        # print('subseq_start-5:', subseq_start, 'subseq_end:', subseq_end)
        if subseq_start < l[0] < subseq_end or subseq_start < l[1] < subseq_end:
        # if (subseq_start-5 < l[0] < subseq_end-5 and subseq_start+5 < l[1] < subseq_end+5):
            gts.append(l)
        # break ## debugging

    print('detection:', detection)
    print('gts:', gts)
    correct_pred, rest_pred, y_pred, y_true, false_neg = ei.get_correct_detections(detection, gts)
    print(correct_pred, rest_pred, false_neg)
    print('y_pred, y_true:', y_pred, y_true,)

    y_pred_all.extend(y_pred)
    y_true_all.extend(y_true)

    all_detections += [(test_file, detection, label_file)]  ### used to plot detections
    all_features += [feature]
    all_tp += [(test_file, correct_pred, label_file)]
    all_fp += [(test_file, rest_pred, label_file)]
    all_fn += [(test_file, false_neg, label_file)]
    all_gt += [(test_file, gts, label_file)]

    # print('all_detections:', all_detections)
    # print('all_features:', all_features)

    print('')
    # break
    


In [ ]:
subseq_start-5 < l[1] < subseq_end-5

In [ ]:
all_fp

In [ ]:
all_fn

### If multiple detections peresent for single GT, calculate averege number of detections per GT

In [ ]:
##### calculate number of instances in each ground truth label
inst_key = inst_in_gt_all.keys()
# print('inst_key:', inst_key)
num_anomaly_per_gt_all = []
for ik in inst_key:
    print('key:', ik)
    anomaly_per_gt = inst_in_gt_all[ik]
    # print('anomaly_per_gt:', anomaly_per_gt)
    num_anomaly_per_gt = len(anomaly_per_gt)
    # print('num_anomaly_per_gt:', num_anomaly_per_gt)
    num_anomaly_per_gt_all.append(num_anomaly_per_gt)
    
print('num_anomaly_per_gt_all:', num_anomaly_per_gt_all)

In [ ]:
y_pred_all

In [ ]:
### Evaluation metrics

from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score, average_precision_score, ConfusionMatrixDisplay, adjusted_rand_score, normalized_mutual_info_score


# Calculate precision
precision = precision_score(y_true_all, y_pred_all)
print(f'Precision: {precision:.4f}')

# Calculate recall
recall = recall_score(y_true_all, y_pred_all)
print(f'Recall: {recall:.4f}')

# # Calculate average precision
# average_precision = average_precision_score(y_true_all, y_pred_all)
# print(f'Average Precision: {average_precision:.4f}')

# Calculate F1 score
f1 = f1_score(y_true_all, y_pred_all)
print(f"F1 Score: {f1:.4f}")
print('')

### isntance length mean and std
mean_inst_len = np.mean(inst_len_all)
std_inst_len = np.std(inst_len_all)
print('avg_inst_len:', mean_inst_len)
print('std_inst_len:', std_inst_len)
print('')

### avg number of anomalies in each detection 
mean_gt_in_inst = np.mean(gt_in_inst_all)
std_gt_in_inst = np.std(gt_in_inst_all)
print('avg_gt_in_inst:', mean_gt_in_inst)
print('std_gt_in_inst:', std_gt_in_inst)
print('')

# ### avg number of detecions per GT
# mean_inst_in_gt = np.mean(num_anomaly_per_gt_all)
# std_inst_in_gt = np.std(num_anomaly_per_gt_all)
# print('mean_inst_in_gt:', mean_inst_in_gt)
# print('std_inst_in_gt:', std_inst_in_gt)
# print('')


# Calculate confusion matrix
conf_matrix = confusion_matrix(y_true_all, y_pred_all)
print("Confusion Matrix:")
print(conf_matrix)
if len(conf_matrix) == 1:
    conf_matrix = np.array([[0, 0], [0, conf_matrix[0][0]]])
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=['normal', 'anomaly'])
disp.plot()

In [ ]:
detection

In [ ]:
# end_mem = process.memory_info().rss / (1024 * 1024)
# print(f"Memory used: {end_mem - start_mem:.2f} MB")

# end_time = time.perf_counter()
# elapsed_ms = (end_time - start_time) * 1000
# print(f"\nTime taken: {elapsed_ms:.2f} ms")

## Classwise Detections

In [ ]:
classwise_fn = defaultdict(list)
classwise_tp = defaultdict(list)
gt_len = 0
for file_fn, file_gt in zip(all_fn, all_gt):
    fn = file_fn[1]
    gt = file_gt[1]
    print('fn:', fn)
    print('gt:', gt)
    for label in gt:
        print('label:', label)
        if label in fn:
            classwise_fn[label[4]].append(label)
        else:
            classwise_tp[label[4]].append(label)
            # print('tp:', label)

    gt_len += len(gt)
    # print('file gt:', len(gt))
    # print('file fn:', len(fn))
    # print('\n')
    # break

total_fn = 0
total_tp = 0
keys = set(list(classwise_fn.keys()) + list(classwise_tp.keys()))
# print('keys:', keys)
class_recall = []
for key in keys:
    print('class:', key)
    total_fn += len(classwise_fn[key])
    total_tp += len(classwise_tp[key])

    crecall = len(classwise_tp[key])/(len(classwise_fn[key])+len(classwise_tp[key]))
    crecall = round(crecall, 4)

    # print('not detected:', len(classwise_fn[key]))
    print('detected:', len(classwise_tp[key]))
    print('total anomalies:', len(classwise_fn[key])+len(classwise_tp[key]))
    print('Recall (classwise):', crecall)
    print('\n')

    class_recall.append(crecall)


# print('total fn+tp:', total_fn+total_tp)
# print('total gt:', gt_len)
assert total_fn+total_tp == gt_len, 'total fn+tp not equal to total gt'
print('All class recalls:', class_recall)

### Store Seperated Instances

In [ ]:
# ### Store Seperated Instances

# ### number of events to take before and after the detection for cropping subsequence
# BUFFER_EVENTS = 0

# total_subseq_lens = [] 
# subseq_class = defaultdict(list)
# new_all_detections = []
# for (test_data, detections, test_label), features in zip(all_detections[0:], all_features):
#     print(test_data, test_label)

#     ### read traces
#     trace = read_traces(test_data)
#     print('trace:', len(trace))    

#     ### path for sub-sequences
#     subseq_path = os.path.dirname(test_label).replace('labels', 'diag_sepAP2_m2/subseq')
#     feature_path = os.path.dirname(test_label).replace('labels', 'diag_sepAP2_m2/feature')
#     # print(subseq_path)

#     ### rules for subsequence
#     # print('detections:', detections)
#     timestamps = [x[1] for x in trace]
#     timestamps = np.array(timestamps)
#     # print('timestamps:', timestamps)

#     ### load ground truths
#     ground_truth_raw = read_traces(test_label)
#     ground_truth = ground_truth_raw['labels']
#     label_trace_name = list(ground_truth.keys())[0]
#     ground_truth = ground_truth[label_trace_name]
#     # print('ground truths:', ground_truth)
#     # print('detections:', detections)
#     # print('features:', features)
#     # print(len(ground_truth))
#     print(len(detections))

#     all_subseq = []
#     new_detections = []
#     for det, feat in zip(detections, features):
#         print('detection:', det)
#         print('feature:', feat)
#         var, ts, file_name = det
#         lb_det, ub_det = ts
#         print('ts:', ts)

#         # print('bounds:', lb_det, ub_det)
#         lb_rel_ts = [abs(x-lb_det) for x in timestamps]
#         # print('lb_rel_ts:', lb_rel_ts)
#         lb_det_ind = np.argmin(lb_rel_ts)
#         # print('lb_trace ind:', lb_det_ind)
#         # print('lb_trace:', timestamps[lb_det_ind])

#         ub_rel_ts = [abs(x-ub_det) for x in timestamps]
#         # print('ub_rel_ts:', ub_rel_ts)
#         ub_det_ind = np.argmin(ub_rel_ts)
#         # print('ub_trace ind:', ub_det_ind)
#         # print('ub_trace:', timestamps[ub_det_ind])

#         # ### exact match using numpy (alternate implementation)
#         # ub_trace = np.where(timestamps == ub_det)[0][0]
#         # print('ub_trace ind:', ub_trace)
#         # print('ub_trace:', timestamps[ub_trace])


#         # lb_trace_ind = np.clip(lb_det_ind - BUFFER_EVENTS, 0, None)
#         lb_trace_ind = lb_det_ind
#         ub_trace_ind = np.clip(ub_det_ind + BUFFER_EVENTS, ub_det_ind, len(trace))
#         subseq_len = ub_trace_ind-lb_trace_ind

#         if subseq_len < 2:
#             # lb_trace_ind = np.clip(lb_trace_ind - (2-subseq_len), 0, None)
#             ub_trace_ind = np.clip(ub_trace_ind + 2, None, len(trace))

#         print('lb_trace ind:', lb_trace_ind)
#         print('ub_trace ind:', ub_trace_ind)

#         print('len of subseq:', ub_trace_ind-lb_trace_ind)
#         sub_seq = trace[lb_trace_ind:ub_trace_ind+1]
#         # print('sub_seq:', sub_seq)
#         # print('feat:', feat)

#         ### update the timestamps for new indices and detections
#         lb_det = timestamps[lb_trace_ind]
#         ub_det = timestamps[ub_trace_ind]
#         # print('lb_det:', lb_det)
#         # print('ub_det:', ub_det)
#         ts = (int(lb_det), int(ub_det))
#         print('new ts:', ts)
#         new_detections.append([var, ts, file_name])

#         all_subseq.append(sub_seq)

#         ### save subsequence
#         sub_seq_name = os.path.basename(test_data)+'_'+str(lb_trace_ind)+'-'+str(ub_trace_ind)+'.json'
#         file_name = sub_seq_name.strip('.json')
#         sub_seq_name = os.path.join(subseq_path,sub_seq_name)
#         if not os.path.exists(subseq_path):
#             os.makedirs(subseq_path)

#         ### save feature
#         feat_name = os.path.basename(test_data)+'_'+str(lb_trace_ind)+'-'+str(ub_trace_ind)+'.json'
#         feat_name = os.path.join(feature_path, feat_name)
#         # print('feat_name:', feat_name)
#         if not os.path.exists(feature_path):
#             os.makedirs(feature_path)
        

#         ### get labels for subsequence
#         no_gt = True
#         for gt in ground_truth:
#             # print('gt:', gt)
#             gt_ind1, gt_ind2 = gt[0], gt[1]
#             # print('gt:', gt_ind1, gt_ind2)
#             # print('lb_trace:', lb_trace_ind, 'ub_trace:', ub_trace_ind)
#             if (gt_ind1 >= lb_trace_ind and gt_ind1 < ub_trace_ind) or (gt_ind2 > lb_trace_ind and gt_ind2 <= ub_trace_ind) or (gt_ind1 < lb_trace_ind < gt_ind2) or (gt_ind1 < ub_trace_ind < gt_ind2):
#                 subseq_class[file_name] += [gt[4]]
#                 no_gt = False
            
#         if no_gt:
#             # subseq_class[file_name] += [100]   ### 100 is the label for FP detections
#             # dont save the subseq
#             pass
#         else:
#             save_json(sub_seq, sub_seq_name)
#             print('subseq:', sub_seq_name)
#             save_json(feat, feat_name)
#             print('feat:', feat_name)

#         ### average length of subsequence
#         total_subseq_lens += [len(sub_seq)]
#         if len(sub_seq) == 0:
#             break

        
#     new_all_detections.append([test_data, new_detections, test_label])
#     print('')
#     # break

# print('subseq_class:', subseq_class)
# ### save the subsequence class labels
# subseq_class_path = os.path.join(subseq_path, 'subseq_labels')
# if not os.path.exists(subseq_class_path):
#     os.makedirs(subseq_class_path)
# save_json(subseq_class, os.path.join(subseq_class_path, 'subseq_class.json'))


In [ ]:
# for file, instances, inst_time, feature in zip(test_files, anomaly_instances, anomaly_timestamps, anomaly_features):
#     print(file)
#     print(instances)
#     print(inst_time)
#     print(feature)

#     file_name = file.split('/')[-1].split('.')[0].split('_')
#     file_name = '_'.join(file_name[:-1])
#     print('file_name 3:', file_name)

#     ### load ground truths
#     test_label = os.path.join(test_labels_basepath, file_name+'_labels.json')
#     print('test_label:', test_label)

#     ground_truth_raw = read_traces(test_label)
#     ground_truth = ground_truth_raw['labels']
#     label_trace_name = list(ground_truth.keys())[0]
#     ground_truth = ground_truth[label_trace_name]
#     print('ground truths:', ground_truth)
#     print(len(ground_truth))

In [ ]:
# keys = list(subseq_class.keys())
# print('keys:', keys)

In [ ]:
#### get detections in correct format for viz
filewise_detections = defaultdict(list)
filewise_tp = defaultdict(list)
filewise_fp = defaultdict(list)
filewise_fn = defaultdict(list)
# all_detections = new_all_detections.copy()
for detection in all_detections:
    # print('detection:', detection)
    # print(detection[1])
    file_name = detection[2]
    det = detection[1]
    for d in det:
        # print('d:', d)
        filewise_detections[file_name].append(d)

for tp in all_tp:
    file_name = tp[2]
    t1 = tp[1]
    for t in t1:
        filewise_tp[file_name].append(t)

for fp in all_fp:
    file_name = fp[2]
    f1 = fp[1]
    for f in f1:
        filewise_fp[file_name].append(f)

for fn in all_fn:
    file_name = fn[2]
    f1 = fn[1]
    for f in f1:
        filewise_fn[file_name].append(f)
    

adapted_detections = []
adapted_tp = []
adapted_fp = []
adapted_fn = []
for file_name in filewise_detections.keys():
    # print('file_name:', file_name)

    for label_path in eval_labels_path:
        if file_name in label_path:
            break
    # print('label_path:', label_path)  

    test_file_path = os.path.join(faultybase_path, file_name)
    # print('test_file_path:', test_file_path)

    adapted_detections.append((test_file_path, filewise_detections[file_name], label_path))
    adapted_tp.append((test_file_path, filewise_tp[file_name], label_path))
    adapted_fp.append((test_file_path, filewise_fp[file_name], label_path))
    adapted_fn.append((test_file_path, filewise_fn[file_name], label_path))

    # break

og_detection = all_detections.copy()
og_tp = all_tp.copy()
og_fp = all_fp.copy()
og_fn = all_fn.copy()

all_detections = adapted_detections
all_tp = adapted_tp
all_fp = adapted_fp
all_fn = adapted_fn



## Save Detections

In [ ]:
all_tp

In [ ]:
# ######## save detections for the dashboard to plot #############
# import traceback

# for test_data, detections, test_label in all_detections:
#     # print(test_data, test_label)
#     # print(test_label.replace('labels', 'detections'))
#     detection_path = test_label.replace('labels', f'diag2_detections')
#     detection_path = detection_path.replace('diag2_detections.json', f'diag2_detections_{DIFF_VAL}.json')
#     # tp_detection_path = detection_path.replace('ei_detections.json', f'tp_ei_detections_{DIFF_VAL}.json')
#     # fp_detection_path = detection_path.replace('ei_detections.json', f'fp_ei_detections_{DIFF_VAL}.json')
#     # print(detections)
#     # print(detection_path)

#     detection_dir = os.path.dirname(detection_path)
#     print(detection_dir)
#     if not os.path.exists(detection_dir):
#         os.makedirs(detection_dir)
#         print(f'Created Directory: {detection_dir}')

#     try:
#         with open(detection_path, 'w') as f:
#             json.dump(detections, f)
#             print(f'Saved detections in {detection_path}')

            
#     except Exception as e:
#         traceback.print_exception(e)
#         print('Error in saving detections')
#         raise ValueError('Error in saving detections')

# for test_data, detections, test_label in all_tp:
#     # print(test_data, test_label)
#     # print(test_label.replace('labels', 'detections'))
#     detection_path = test_label.replace('labels', 'diag2_detections')
#     tp_detection_path = detection_path.replace('diag2_detections.json', f'tp_diag2_detections_{DIFF_VAL}.json')
#     # fp_detection_path = detection_path.replace('ei_detections.json', 'fp_ei_detections.json')
#     # print(detections)

#     detection_dir = os.path.dirname(detection_path)
#     # print(detection_dir)
#     if not os.path.exists(detection_dir):
#         os.makedirs(detection_dir)
#         print(f'Created Directory: {detection_dir}')

#     try:

#         with open(tp_detection_path, 'w') as f:
#             json.dump(detections, f)
#             print(f'Saved detections in {tp_detection_path}')
            
#     except Exception as e:
#         traceback.print_exception(e)
#         print('Error in saving detections')
#         raise ValueError('Error in saving detections')


# for test_data, detections, test_label in all_fp:
#     # print(test_data, test_label)
#     # print(test_label.replace('labels', 'detections'))
#     detection_path = test_label.replace('labels', 'diag2_detections')
#     # tp_detection_path = detection_path.replace('ei_detections.json', 'tp_ei_detections.json')
#     fp_detection_path = detection_path.replace('diag2_detections.json', f'fp_diag2_detections_{DIFF_VAL}.json')
#     # print(detections)

#     detection_dir = os.path.dirname(detection_path)
#     # print(detection_dir)
#     if not os.path.exists(detection_dir):
#         os.makedirs(detection_dir)
#         print(f'Created Directory: {detection_dir}')

#     try:

#         with open(fp_detection_path, 'w') as f:
#             json.dump(detections, f)
#             print(f'Saved detections in {fp_detection_path}')
            
#     except Exception as e:
#         traceback.print_exception(e)
#         print('Error in saving detections')
#         raise ValueError('Error in saving detections')
        